In [1]:
import time
import requests
import json
import multiprocessing as mp
import mag_functions as F
from math import ceil

In [3]:
# input that contains a single affiliation string to parse
with open("test_json_single.json", "rb") as f:
    input_json = json.load(f)

In [48]:
# input that contains hard samples to make sure the model does not return errors
# with open("test_json_single_empty.json", "rb") as f:
#     input_json = json.load(f)

In [11]:
# input that contains 6 affiliation strings to parse
# with open("test_json_batch.json", "rb") as f:
#     input_json = json.load(f)

### Testing Through Sagemaker

In [1]:
import boto3
import json

In [22]:
sagemaker = boto3.client("sagemaker-runtime", region_name="us-east-1")

endpoint_name='openalex-topic-classifier-v1-endpoint'

In [52]:
%%time
response = sagemaker.invoke_endpoint(
                            EndpointName=endpoint_name, 
                            ContentType='application/json',
                            Body=bytes(json.dumps(input_json), 'utf-8'))

CPU times: user 34.9 ms, sys: 0 ns, total: 34.9 ms
Wall time: 422 ms


In [21]:
response['Body'].read().decode()

'[[{"topic_id": 10829, "topic_label": "829: Networks on Chip in System-on-Chip Design", "topic_score": 0.9978}, {"topic_id": 10054, "topic_label": "54: Parallel Computing and Performance Optimization", "topic_score": 0.9963}, {"topic_id": 11522, "topic_label": "1522: Design and Optimization of Field-Programmable Gate Arrays and Application-Specific Integrated Circuits", "topic_score": 0.991}]]'

### Testing Through API

In [98]:
%%time
# testing the call to the API one time
len(json.loads(F.get_tags(input_json, 1)[1]))

CPU times: user 36.2 ms, sys: 0 ns, total: 36.2 ms
Wall time: 956 ms


1

In [12]:
json.loads(F.get_tags(input_json, 1)[1])

[[{'topic_id': 10829,
   'topic_label': '829: Networks on Chip in System-on-Chip Design',
   'topic_score': 0.9978},
  {'topic_id': 10054,
   'topic_label': '54: Parallel Computing and Performance Optimization',
   'topic_score': 0.9963},
  {'topic_id': 11522,
   'topic_label': '1522: Design and Optimization of Field-Programmable Gate Arrays and Application-Specific Integrated Circuits',
   'topic_score': 0.991},
  {'topic_id': 12923,
   'topic_label': '2923: Connected Component Labeling Algorithms',
   'topic_score': 0.9897},
  {'topic_id': 12292,
   'topic_label': '2292: Graph Matching and Analysis Techniques',
   'topic_score': 0.9872}]]

### Test throughput with Locust

In [ ]:
!locust --processes 4

### Testing API with data random queried OpenAlex data

In [ ]:
%%time
open_req = "https://api.openalex.org/works/random"
resp = requests.get(open_req).json()
print(resp['id'])

In [ ]:
%%time
open_req = "https://api.openalex.org/works/W4205779344"
resp = requests.get(open_req).json()
print(resp['id'])

if resp['primary_location']['source']:
    journal_display_name = resp['primary_location']['source']['display_name']
else:
    journal_display_name = ""


input_json = [{'title': resp['title'], 
               'abstract_inverted_index': resp['abstract_inverted_index'], 
               'journal_display_name': journal_display_name, 
               'referenced_works': resp['referenced_works'],
               'inverted': True}]

model_res = sagemaker.invoke_endpoint(
        EndpointName='openalex-topic-classifier-v1-endpoint',
        Body=bytes(json.dumps(input_json), 'utf-8'),
        ContentType='application/json',
        Accept='Accept')
print(input_json[0]['title'])
print("")
for i in json.loads(model_res['Body'].read().decode())[0]:
    print(i)